In [1]:
!pip3 install arrow

import os
import arrow
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
os.listdir('/content/gdrive/My Drive/vc_modeling')

['presentation_poster.pptx',
 'data',
 'regression_targets',
 'feature_extraction',
 'model output data spec.gsheet',
 'models',
 '.ipynb_checkpoints',
 'model_output']

In [5]:
file_names = os.listdir("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/")
print(file_names)

['category_groups.csv', 'funding_rounds.csv', 'people.csv', 'checksum.csv', 'people_descriptions.csv', 'investors.csv', 'organization_descriptions.csv', 'investment_partners.csv', 'event_appearances.csv', 'organizations.csv', 'org_parents.csv', 'jobs.csv', 'acquisitions.csv', 'funds.csv', 'ipos.csv', 'degrees.csv', 'investments.csv', 'events.csv']


In [8]:
funding_df = pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/funding_rounds.csv")
funding_df.head(2)

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,143236.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,500000.0,USD,NaN,NaN,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,145939.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36


Classification features

In [ ]:
num_funding_rounds = funding_df.groupby("org_uuid", as_index="False").size().reset_index(name="num_funding_rounds")
num_funding_rounds.head()

,org_uuid,num_funding_rounds
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,1
1,000014da-0c46-b9cb-0941-3a93c027b119,7
2,00002470-bff7-6226-5800-0ca1b3787b6f,5
3,000095de-8e2b-82f1-32a7-c222ba3d5682,1
4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,2


In [ ]:
avg_funding_amt_usd = funding_df.groupby("org_uuid", as_index="False").mean(numeric_only=True)[["raised_amount_usd"]].dropna(how="all", subset=["raised_amount_usd"])
avg_funding_amt_usd.head()

,raised_amount_usd
org_uuid,
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,8.260736e+07
00002470-bff7-6226-5800-0ca1b3787b6f,1.133333e+06
000095de-8e2b-82f1-32a7-c222ba3d5682,9.500000e+06
0000d497-c93a-eea3-eeb0-a943dfb4f71e,4.315760e+05
0001a8cc-0cdc-4a30-b4d3-da1b425069e1,1.500000e+06


In [ ]:
funding_features = num_funding_rounds.join(avg_funding_amt_usd, on="org_uuid")
funding_features[:100]

,org_uuid,num_funding_rounds,raised_amount_usd
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,1,8.260736e+07
1,000014da-0c46-b9cb-0941-3a93c027b119,7,NaN
2,00002470-bff7-6226-5800-0ca1b3787b6f,5,1.133333e+06
3,000095de-8e2b-82f1-32a7-c222ba3d5682,1,9.500000e+06
4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,2,4.315760e+05
...,...,...,...
95,00295b25-f235-462c-a096-90eee885b296,1,NaN
96,002aac96-79d4-a8f3-8ef0-d44fb240951d,1,3.000000e+06
97,002ad549-e548-4dcd-9eb6-1029c7590c4b,1,2.236403e+06
98,002b904e-7583-3ab0-3151-d130e0db7561,1,1.000000e+03


In [ ]:
from google.colab import files

# export features to csv
funding_features.to_csv('funding_features.csv', columns = ["org_uuid", "num_funding_rounds", "raised_amount_usd"], index=False) 
files.download('funding_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Time-variant regression features

In [26]:
acquisitions_df = pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/acquisitions.csv")
ipos_df = pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/ipos.csv")


funding_data = funding_df[['org_uuid', 'announced_on', 'post_money_valuation_usd']].rename(columns={"announced_on": "date", "post_money_valuation_usd": "valuation"})
acquisitions_data = acquisitions_df[['acquiree_uuid', 'acquired_on', 'price_usd']].rename(columns={"acquiree_uuid": "org_uuid", "acquired_on": "date", "price_usd": "valuation"})
ipos_data = ipos_df[['org_uuid', 'went_public_on', 'valuation_price_usd']].rename(columns={"went_public_on": "date", "valuation_price_usd": "valuation"})

types = acquisitions_df[['type']].values
acquisition_types = acquisitions_df[['acquisition_type']].values
acquisitions_data['event'] = [types[x][0] if not isinstance(acquisition_types[x][0], str) 
                              else acquisition_types[x][0] for x in range(len(acquisitions_data))]
funding_data['event'] = pd.Series([x[0] for x in funding_df['name'].str.split(" - ")])
ipos_data['event'] = ipos_df['stock_exchange_symbol']

timelines = pd.concat([funding_data, acquisitions_data, ipos_data])

# Normalize dates

timelines.sort_values(by=['org_uuid', 'date'], inplace=True)
timelines.reset_index(inplace=True, drop=True)
timelines = timelines[timelines['valuation'].notnull()]  # We only care about times at which there is a valuation of the company.
timelines.set_index(['org_uuid'])
grouped_timelines = timelines.groupby(['org_uuid'])
timeline_counts = grouped_timelines.count().reset_index()
orgs_with_2plus_vals = timeline_counts[timeline_counts['date']>1].org_uuid.unique()  # at least 2 valuations
timelines = timelines[timelines['org_uuid'].isin(orgs_with_2plus_vals)]

milli_per_day = 86400  # Used for converting Unix time

regression_df = pd.DataFrame()
for company_id in timelines.org_uuid.unique():
  timeline = grouped_timelines.get_group(company_id).copy()
  timeline.reset_index(inplace=True, drop=True)
  if timeline.empty:
    continue

  # print(timeline['date'])
  day_zero = datetime.datetime.strptime(timeline['date'].iloc[0], '%Y-%m-%d').timestamp()
  parsed_dates = pd.Series([datetime.datetime.strptime(x, '%Y-%m-%d').timestamp() for x in timeline['date']])
  norm_dates = (parsed_dates - day_zero)/milli_per_day
  timeline['norm_dates'] = norm_dates
  regression_df = regression_df.append(timeline)

regression_df.head(2)

,org_uuid,date,valuation,event,norm_dates
0,0070da44-2860-2901-9ace-4c633fb657b2,1996-09-13,7.000000e+09,acquisition,0.0
1,0070da44-2860-2901-9ace-4c633fb657b2,2014-11-13,4.700000e+09,acquisition,6635.0


In [27]:
org_launches = regression_df.groupby('org_uuid').min().reset_index()[['org_uuid', 'date']]
org_launch_dict = dict(zip(org_launches.org_uuid.values, org_launches.date.values))

In [43]:
def get_org_funding_features(org_uuid, days_forward):

  org_launch_date = org_launch_dict[org_uuid]
  days_forward_date = arrow.get(org_launch_date).shift(days=days_forward).format('YYYY-MM-DD')
  df = funding_df[(funding_df['org_uuid'] == org_uuid) & (funding_df['announced_on'] <= days_forward_date)]
  
  # num funding rounds
  num_funding_rounds = df.groupby("org_uuid", as_index="False").size().reset_index(name="num_funding_rounds")

  # funding amount usd
  avg_funding_amt_usd = df.groupby("org_uuid", as_index="False").mean(numeric_only=True)[["raised_amount_usd"]].dropna(how="all", subset=["raised_amount_usd"])
  
  return num_funding_rounds.merge(avg_funding_amt_usd, on=['org_uuid'])
  

In [45]:
org_uuid = 'df662812-7f97-0b43-9d3e-12f64f504fbb'  # facebook
days_forward = 200

get_org_funding_features(org_uuid, days_forward)

,org_uuid,num_funding_rounds,raised_amount_usd
0,df662812-7f97-0b43-9d3e-12f64f504fbb,2,6600000.0


In [49]:
DAYS_FORWARD_OPTIONS = [200, 500, 1000, 2000]

features_by_days_forward = pd.DataFrame()

for org_uuid in regression_df.org_uuid.unique():
  for days_forward in DAYS_FORWARD_OPTIONS:
    org_features = get_org_funding_features(org_uuid, days_forward)
    org_features['days_forward'] = days_forward
    features_by_days_forward = features_by_days_forward.append(org_features)

features_by_days_forward.to_csv("/content/gdrive/My Drive/vc_modeling/data/funding_features_for_regression.csv", index=False)
features_by_days_forward.head(10)

,org_uuid,num_funding_rounds,raised_amount_usd,days_forward
0,007bfe46-e02c-00b3-51ad-3a25e8d64851,1,2.458700e+05,200
0,007bfe46-e02c-00b3-51ad-3a25e8d64851,3,8.766840e+05,500
0,007bfe46-e02c-00b3-51ad-3a25e8d64851,4,1.155582e+06,1000
0,007bfe46-e02c-00b3-51ad-3a25e8d64851,5,1.140286e+06,2000
0,009ae60e-6dec-7a8c-8382-2edccb0f9cce,1,6.411100e+04,200
0,009ae60e-6dec-7a8c-8382-2edccb0f9cce,3,1.090555e+05,500
0,009ae60e-6dec-7a8c-8382-2edccb0f9cce,4,1.118563e+05,1000
0,009ae60e-6dec-7a8c-8382-2edccb0f9cce,4,1.118563e+05,2000
0,00bf570a-fca7-47bc-97c3-578f1be0069e,1,6.500000e+05,200
0,00bf570a-fca7-47bc-97c3-578f1be0069e,1,6.500000e+05,500
